In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import openai
df=pd.read_csv("/content/faq_data.csv")
print(df)

              topic                                           question  \
0  Airport Services  What types of baggage services are available a...   
1  Airport Services  How can I check in for my flight online or at ...   
2  Airport Services  What are the airport's security procedures and...   
3  Airport Services  Where can I find information about airport tra...   
4  Airport Services  Are there any services available for passenger...   
5  Airport Services  What amenities and facilities are available at...   
6  Airport Services  How can I report lost and found items at the a...   
7  Airport Services    What are the airport's policies regarding pets?   
8  Airport Services  How can I contact airport customer service or ...   

                                              answer  
0  Airline X offers several baggage services at t...  
1  **Online Check-in:**\n\nTo check in online, vi...  
2  Airline X does not control airport security pr...  
3  Airline X does not provide airport t

In [5]:
from openai import OpenAI
from google.colab import userdata
#Feel free to use chatgpt or other models - Google offere free gemini api hence I have used this
openai = OpenAI(
   base_url="https://generativelanguage.googleapis.com/v1beta/",
  api_key=userdata.get('gemini_api'),
)

1045

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


# Optional: chunk answers into smaller pieces (e.g. 200 tokens) for better retrieval
def chunk_text(text, max_tokens=200):
    words = text.split()
    for i in range(0, len(words), max_tokens):
        yield " ".join(words[i:i+max_tokens])

chunks = []
metadata = []
for idx, row in df.iterrows():
    for c in chunk_text(row['answer']):
        chunks.append(c)
        metadata.append({
            "topic": row['topic'],
            "question": row['question'],
            "answer_chunk": c
        })


In [9]:
from sentence_transformers import SentenceTransformer
import numpy as np
#I am using the huggingface model to generate embeddings , If the data was large once can try & explore other embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Embed chunks
chunk_vectors = embedder.encode(chunks, convert_to_numpy=True)



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Build FAISS index as before
import faiss

dimension = chunk_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_vectors)

# For query embedding function
def embed_query(query):
    return embedder.encode([query], convert_to_numpy=True)[0]


In [11]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['question'])
def retrieve_similar_answer_chunks(query, k=5):
    q_vec = embed_query(query)
    D, I = index.search(np.array([q_vec]).astype('float32'), k)
    return [metadata[i] for i in I[0]]

def retrieve_similar_questions(query, k=5):
    q_tfidf = tfidf_vectorizer.transform([query])
    scores = (tfidf_matrix @ q_tfidf.T).toarray()
    topk_idx = scores[:,0].argsort()[-k:][::-1]
    return df.iloc[topk_idx][['topic', 'question', 'answer']].to_dict(orient='records')


In [12]:
def hybrid_retrieve(query, k=5):
    answer_chunks = retrieve_similar_answer_chunks(query, k)
    question_hits = retrieve_similar_questions(query, k)
    combined_contexts = answer_chunks + question_hits

    # Deduplicate if needed
    seen = set()
    filtered = []
    for c in combined_contexts:
        key = (c.get('topic'), c.get('question'))
        if key not in seen:
            filtered.append(c)
            seen.add(key)
    return filtered



In [13]:
def generate_answer(query):
    contexts = hybrid_retrieve(query, k=5)
    context_text = "\n\n".join([f"Q: {c['question']}\nA: {c.get('answer') or c.get('answer_chunk')}" for c in contexts])

    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert airline assistant answering user queries based on provided context."
                " Use the context to generate a helpful, factual, self-contained answer."
                "If the context doesn't help & you are unable to answer - Please reply to the user to reach out to our customer service call center at 1 800 800 000 or email us at customercare@airlinex.com"
                "If the user is asking some random question not related to be asked to an airline assistant reply - I don't understand your question can you please rephrase"
            )
        },
        {
            "role": "user",
            "content": f"Context:\n{context_text}\n\nQuestion: {query}\nAnswer:"
        }
    ]

    response = openai.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages,
        temperature=0.7,
        max_tokens=700,
    )
    return response.choices[0].message.content

Now lets test out for few queries & see if the agent is performing as expected or not

In [14]:
query = "Can you change my booking date?"
print(generate_answer(query))


I am sorry, I cannot help you with changing your booking date.  Please contact our customer service call center at 1 800 800 000 or email us at customercare@airlinex.com



In [15]:
query = "Can I go to mars"
print(generate_answer(query))

I don't understand your question can you please rephrase



In [16]:
query="What service are offered for people with special needs"
print(generate_answer(query))

Airline X offers several services for passengers with disabilities or special needs.  These include pre-booking assistance for wheelchair requests, special meal requests for medical dietary needs, and priority boarding.  For passengers needing additional assistance (mobility help, oxygen, etc.), contact the special assistance department at least 48 hours before departure.  Specific services may vary by destination and aircraft.  More details are in our passenger guide or by contacting the special assistance department.



In [17]:
# To understand how and what hybrid_retriever did we can check which documents were retruned from here
hybrid_retrieve(query)

[{'topic': 'Airport Services',
  'question': 'Are there any services available for passengers with disabilities or special needs?',
  'answer_chunk': 'Airline X is committed to providing a safe and comfortable travel experience for all passengers, including those with disabilities or special needs. We offer a range of services to assist you throughout your journey. These include pre-booking assistance with wheelchair requests, special meal requests to accommodate dietary restrictions related to medical conditions, and priority boarding to allow ample time for settling in. For passengers requiring additional assistance, such as those needing help with mobility or oxygen, we recommend contacting our special assistance department at least 48 hours prior to departure to arrange the necessary support. Our staff is trained to provide courteous and efficient service to meet individual needs. Specific services and limitations may vary depending on the destination and aircraft type. Detailed in